**Introduction to Computer Theory, 2nd. Edition by Daniel Cohen**

CSCI-549 East Texas A&M University

Lon Cherryholmes, Sr.

In [ ]:
!pip install SNOBOL4python==0.4.5
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz
!pip install networkx

In [ ]:
import re
from graphviz import Digraph
from itertools import product
from pprint import pformat, pprint
from IPython.display import display, Latex, Markdown
import sys
from pprint import pprint, pformat
## Thirty one (31) flavors of patterns to choose from ...
from SNOBOL4python import ε, σ, π, λ, Λ, ζ, θ, Θ, φ, Φ, α, ω
from SNOBOL4python import ABORT, ANY, ARB, ARBNO, BAL, BREAK, BREAKX, FAIL
from SNOBOL4python import FENCE, LEN, MARB, MARBNO, NOTANY, POS, REM, RPOS
from SNOBOL4python import RTAB, SPAN, SUCCESS, TAB
# Miscellaneous
from SNOBOL4python import GLOBALS, TRACE, PATTERN, STRING
from SNOBOL4python import ALPHABET, DIGITS, UCASE, LCASE, NULL
from SNOBOL4python import nPush, nInc, nPop, Shift, Reduce, Pop
# Instantiate the global variable space
GLOBALS(globals())
FA = dict()

In [ ]:
def display_dfa(fa):
    dot = Digraph()
    dot.attr(rankdir='LR')
    dot.attr(bgcolor='black')
    dot.attr('node', style='filled', fillcolor='black', fontcolor='white', color='white')
    dot.attr('edge', color='white', fontcolor='white')
    Σ = fa[0][1:]
    Δ = fa[1:]
    for i, δ in enumerate(Δ):
        q = δ[0]
        is_start = False
        is_finish = False
        if m := re.match(r"^\-(.*)$", q): is_start = True;  q = m.groups(1)[0]
        if m := re.match(r"^\+(.*)$", q): is_finish = True; q = m.groups(1)[0]
        if is_finish: dot.node(str(i+1), shape='doublecircle')
        else:         dot.node(str(i+1), shape='circle')
        if is_start:
            dot.node('', width='0', height='0', margin='0', shape='point', style='invis')
            dot.edge('', str(i+1), )
    for i, δ in enumerate(Δ):
        for j in range(1, len(δ)):
            dot.edge(str(i+1), str(δ[j]), label=Σ[j-1])
    dot.render('dfa', format='png', cleanup=False)  # Saves as dfa.png
    display(dot)

In [ ]:
def compile_fa(fa_table):
    header = fa_table[0][1:]
    states = fa_table[1:]
    start_state = None
    accepting_states = set()
    transitions = {}
    for row in states:
        state_label = row[0]
        state_num = int(state_label.lstrip('+-'))
        if state_label.startswith('-'):
            start_state = state_num
        if state_label.startswith('+') or state_label.startswith('-+'):
            accepting_states.add(state_num)
        transitions[state_num] = {}
        for symbol, next_state in zip(header, row[1:]):
            transitions[state_num][symbol] = next_state
    if start_state is None:
        raise ValueError("No start state defined in FA table.")
    return start_state, header, transitions, accepting_states

In [ ]:
def run_fsm(input_str, start_state, header, transitions, accepting_states):
    current_state = start_state
    for ch in input_str:
        if ch not in header:
            raise ValueError(f"Invalid input symbol: {ch}")
        current_state = transitions[current_state][ch]
    accepted = current_state in accepting_states
    return accepted, current_state

In [ ]:
def show_samples(table):
    accepted = []
    rejected = []
    alphabet = table[0][1:]
    fsm = compile_fa(table)
    for length in range(0, 8):
        for combo in product(alphabet, repeat=length):
            word = ''.join(combo)
            is_accepted, _ = run_fsm(word, *fsm)
            if is_accepted:
                accepted.append(word)
            else: rejected.append(word)
    return accepted, rejected

In [ ]:
import textwrap
def display_problem(problem_name, verbose=True):
    problem = FA[problem_name]
    display(Latex(problem[0]))
    display_dfa(problem[2])
    if (verbose):
        accepted, rejected = show_samples(problem[2])
        print("Accepted:")
        print(textwrap.fill(" ".join(accepted), width=80))
        print()
        print("Rejected:")
        print(textwrap.fill(" ".join(rejected), width=80))

---
**Chapter 2: Languages**

In [ ]:
from itertools import product
from pprint import pprint

def kleene_star(S):
    result = [{} for _ in range(8)]
    for repeat in range(8):
        for combo in product(S, repeat=repeat):
            combo = "".join(combo)
            length = len(combo)
            if length < len(result):
                if combo not in result[length]:
                    result[length][combo] = 1
                else: result[length][combo] += 1
    for length, words in enumerate(result):
        if words is not None:
            print(length, len(words), words)

---
**Problem 2.1:** Consider the language $S^{*}$, where $S =$ {*a b*}.
1. How many words does the language have of length 2?
2. of length 3?
3. of length *n*?

**Answer 2.1:**
1. $4$
2. $8$
3. $2^{n}$

In [ ]:
# Observe the powers of two sequence: 1, 2, 4, 8, 16, 32, 64, ...
S = {"a", "b"}
kleene_star(S)

---
**Problem 2.2:** Consider the language $S^{*}$, where $S =$ {*aa b*}.
1. How many words does the language have of length 4?
2. of length 5?
3. of length 6?
4. What can be said in general?

**Answer 2.2:**
1. $5$
2. $8$
3. $13$
4. *See below.**

In [ ]:
# Observe the Fibonacci sequence: 1, 1, 2, 3, 5, 8, 13, ...
S = {"aa", "b"}
kleene_star(S)

---
**Problem 2.3:** Consider the language $S^{*}$, where $S =$ {*ab ba*}.
1. Write out all the words in $S^{*}$ that have seven or fewer letters.
2. Can any word in this language contain the substrings *aaa* or *bbb*?
3. What is the smallest word *not* in this language?

**Answer 2.3:**

1. *See below.*
2. No. $S$ does not contain *aa* or *bb*. Any substring has at most two consecutive *a*'s or *b*'s.
3. Either *a* or *b*.

In [ ]:
# Answer 2.3.1
S = {"ab", "ba"}
kleene_star(S)

---
**Problem 2.4:** Consider the language $S^{*}$, where $S =$ {*a ab ba*}.
1. Is the string (*abbba*) a word in this language?
2. Write out all the words in this language with six or fewer letters.
3. What is another way in which to describe the words in this language? Be careful, this is not simply the language of all words without bbb.

**Answer 2.4:**

1. No. Neither *b* nor *bb* are in $S$. At most two consecutive *b*'s.
2. *See below.*
3. All the strings of *a*'s and *b*'s where each *b* is adjacent to a unique *a*.

In [ ]:
# Answer 2.4.2
S = {"a", "ab", "ba"}
kleene_star(S)

---
**Problem 2.5:** Consider the language $S^{*}$, where $S =$ {*aa aba baa*}.
1. Show that the words *aabaa*, *baaabaaa*, and *baaaaababaaaa* are all in this language.
2. Can any word in this language be interpreted as a string of elements from $S$ in two different ways?
3. Can any word in this language have an odd total number of *a*'s?

**Answer 2.5:**
1. *aabaa* = *aa baa* \
   *baaabaaa* = *baa aba aa* \
   *baaaaababaaaa* = *baa aa aba baa aa*
2. No. None of the words in $S$ is a prefix of another word in $S$. Neither *a*, *ab*, nor *ba* are elements of $S$.
3. No. Each element has two *a*'s and any composite string will have only multiples.

---
**Problem 2.6:** Consider the language $S^{*}$, where $S =$ {*xx xxx*}.
In how many ways can $x^{19}$ be written as the product of words in $S$?
This means: How many different factorizations are there of $x^{19}$ into *xx* and *xxx*?

**Answer 2.6:**

$\text{Let } a = \#(\text{copies of } xx), \quad b = \#(\text{copies of } xxx)$.

The length condition is $2a + 3b = 19, \quad a,b \in \mathbb{Z}_{\ge 0}$.

For each feasible pair $(a,b)$, the number of distinct orderings is $\binom{a+b}{a}$.

Solving $2a + 3b = 19$:

$$
\begin{aligned}
b &= 1 \quad\Rightarrow\quad a = 8, &\binom{9}{8} &= 9, \\
b &= 3 \quad\Rightarrow\quad a = 5, &\binom{8}{5} &= 56, \\
b &= 5 \quad\Rightarrow\quad a = 2, &\binom{7}{2} &= 21.
\end{aligned}
$$

Summing over all solutions: $9 + 56 + 21 = 86$.

$$
\boxed{\text{There are 86 distinct factorizations.}}
$$

---
**Problem 2.7:** Consider the language PALINDROME over the alphabet {*a b*}.

$\textbf{Let}$ $\text{PALINDROME} = \{\, w \in \{a,b\}^* \mid w = w^R \,\}$, where $w^R$ denotes the reverse of $w$.

* (i) Prove that if $x$ is in PALINDROME, then so is $x^{n}$ for any *n*.
* (ii) Prove that if $y^{3}$ is in PALINDROME, then so is $y$.
* (iii) Prove that if $z^{n}$ is in PALINDROME for some $n$ (greater than 0), then $z$ itself is also.
* (iv) Prove that PALINDROME has as many words of length 4 as it does of length 3.
* (v) Prove that PALINDROME has as many words of length $2n$ as it has of length $2n - 1$. How many words is that?

---
**Answer 2.7(i):**

$\textbf{Claim.}$ If $x \in \text{PALINDROME}$, then $x^n \in \text{PALINDROME}$ for any integer $n \ge 0$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $x \in \text{PALINDROME}$, we have $x = x^R$. Thus, $(x^n)^R = (x^R)^n = x^n$.

---
**Answer 2.7(ii):**

$\textbf{Claim.}$ If $y^3 \in \text{PALINDROME}$, then $y \in \text{PALINDROME}$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $y^3 \in \text{PALINDROME}$, we have $yyy = (yyy)^R$, $(yyy)^R = y^Ry^Ry^R$. Given they are one-to-one and corresponding, therefore $y = y^R$.

---
**Answer 2.7(iii):**

$\textbf{Claim.}$ If $y^3 \in \text{PALINDROME}$, then $y \in \text{PALINDROME}$.

$\textbf{Proof.}$

Recall that for any strings *u,v*, we have $(uv)^R = v^R u^R.$

Since $z^n \in \text{PALINDROME}$, we have $z^n = (z^n)^R$, $(z^n)^R = (z^R)^n$. Given they are one-to-one and corresponding, therefore $z = z^R$.

**Answer 2.7(iv):**

For alphabet {*a b*}, a palindrome of length $n$ is determined by its first ${\lceil n/2 \rceil}$ positions, each with 2 choices. Thus the number of palindromes of length $n$ is $2^{\lceil n/2 \rceil}$. For $n=3$ the length is $2$ and for $n=4$ the length is $2$, hence the same number of palindromes, which is $4$.

---
**Answer 2.7(v):**

For alphabet {*a b*}, a palindrome of length $n$ is determined by its first ${\lceil n/2 \rceil}$ positions, each with 2 choices. Thus the number of palindromes of length $n$ is $2^{\lceil n/2 \rceil}$. For $n=2N$ the length is ${\lceil 2N/2 \rceil}$ and for $n=2N-1$ the length is ${\lceil (2N-1)/2 \rceil}$, but $2N/2=(2N-1)/2=N$ hence the same number of palindromes, which is $2^{\lceil N \rceil}$.

---
**Problem 2.8:** Show that if the concatenation of two words (neither $Λ$) in PALINDROME is also a word in PALINDROME, then both words are powers of some other word; that is, if $x$ and $y$ and $xy$ are all in PALINDROME, then there is a word $z$ such that $x = z^{p}$  and $y = z^{q}$ for some integers $p$ and $q$ (maybe $p$ or $q = 1$).

* $x = x^R$
* $y = y^R$
* $(xy) = (xy)^R = (y^Rx^R) = (yx)$
  
so $x$ and $y$ commute.

We prove by induction on $|x|+|y|$ that if nonempty words $x,y$ commute, then there exists $z$ with $x=z^m$ and $y=z^n$ for some $m,n\ge 1$.

If $|x|=|y|$, then $xy=yx$ implies $x=y$, and taking $z=x$ gives the claim. Otherwise, without loss of generality $|x|>|y|$. Since $xy=yx$, the word $y$ is a prefix of $x$; write $x=yt$ for some (possibly empty) $t$. Substituting into $xy=yx$ yields

$(yt)y = y(yt) \;\;\Rightarrow\;\; yty = yyt \;\;\Rightarrow\;\; ty=yt$,

where we used left-cancellation in the free monoid. If $t=\varepsilon$, then $x=y$ and we are done as before. If $t\neq\varepsilon$, then $t$ and $y$ are shorter nonempty words that commute, and $|t|+|y|<|x|+|y|$. By the induction hypothesis, there exists a word $z$ and integers $p,q\ge 1$ such that $t=z^p$ and $y=z^q$. Hence

$x=yt = z^q z^p = z^{p+q} \quad\text{and}\quad y=z^q$,

so taking $m=p+q$ and $n=q$ completes the proof.

Thus, from $x,y,xy\in$ PALINDROME with $x,y\neq\varepsilon$, we conclude that $x$ and $y$ are powers of a common word.

---
**Problem 2.9:**
* (i) Let $S$ = {*ab bb*} and let $T$ = {*ab bb bbbb*}. Show that $S^{*} = T^{*}$.
* (ii) Let $S$ = {*ab bb*} and let $T$ = {*ab bb bbb*}. Show that $S^{*} \neq T^{*}$, but that $S^{*} \subset T^{*}$.
* (iii) What principle does this illustrate?

Proof that $S^{*} = T^{*}$

Let $S =$ {*ab bb*}, $T =$ {*ab bb bbbb*}.

By definition of the Kleene star:

$
A^{0} = \{\varepsilon\},
A^{n} = \{x_{1}x_{2}\cdots x_{n} \mid x_{i} \in A \ \text{for all } i\},
A^{*} = \bigcup_{n \ge 0} A^{n}.
$

1. $S^{*} \subseteq T^{*}$

Let $w \in S^{*}$. Then $w \in S^{n}$ for some $n \ge 0$, so $w = s_{1}s_{2}\cdots s_{n}
\quad\text{with each } s_{i} \in S$.

Since $S \subseteq T$, each $s_{i} \in T$, hence $w \in T^{n} \subseteq T^{*}$.

2. $T^{*} \subseteq S^{*}$

Let $w \in T^{*}$. Then $w \in T^{k}$ for some $k \ge 0$, so $w = x_{1}x_{2}\cdots x_{k} \quad\text{with each } x_{i} \in T = \{ab,\, bb,\, bbbb\}$.

We can rewrite each \(x_{i}\) as a sequence of elements from $S$:
- If $x_{i} = ab$, replace it by $(ab)$.
- If $x_{i} = bb$, replace it by $(bb)$.
- If $x_{i} = bbbb$, note that $bbbb = (bb)(bb)$, so replace it by ($bb$ $bb$).

Concatenating all these replacements yields $w = y_{1}y_{2}\cdots y_{m}\quad\text{with each } y_{j} \in S$.

Thus $w \in S^{m} \subseteq S^{*}$.

3. Since $S^{*} \subseteq T^{*}$ and $T^{*} \subseteq S^{*}$, we have $S^{*} = T^{*}$.

---
**Problem 2.10:** How does the situation in Problem 9 change if we replace the operator $^{*}$ with the operator $^{+}$ as defined in this chapter? Note the language $S^{+}$ means the same same $S^{*}$, but does not allow the "concatenation of no words" of $S$.

**Answer 2.10:**
No changes in the the equalities and inclusions.

---
**Problem 2.11:** Prove that for all sets $S$,
* (i) $(S^{+})^{*} = (S^{*})^{*}$
* (ii) $(S^{+})^{+} = S^{+}$
* (iii) Is $(S^{*})^{+} = (S^{+})^{*}$ for all sets $S$?

**Answer 2.11:**

* (i) $(S^+)^*$ includes $Λ$ even if $S$ does not, so $(S^+)^*=S^*$. $S^*=S^**$ by Theorem 1.
* (ii) There can be no factor in $S$ that is not in $S^+$, $(S^+)^+ \subseteq S^+$. In general any set is contained in its positive closure $S^+ \subseteq (S^+)^+$. Therefore, $(S^+)^+ = S^+$.
* (iii) Yes.

---
**Problem 2.12:** Let $S$ = {*a bb bab abaab*}.
1. Is *abbabaabab* in $S^{*}$?
2. Is *abaabbabbaabb*?
3. Does any word in $S^{*}$ have an odd total number of *b*'s?

**Answer 2.12:**

1. No.
2. No.
3. No.

---
**Problem 2.13:** Suppose that for some language $L$ we can always concatenate two words in $L$ and get another word in $L$, if and only if the words are not the same. That is, for any words $w_{1}$ and $w_{2}$ in L where $w_{1} \neq w_{2}$, the word $w_{1}$$w_{2}$ is in $L$ but the word $w_{1}$$w_{1}$ is not in $L$. Prove that this cannot happen.

**Answer 2.13:** This is the same as saying that the language L would allow all concatenations that did not produce squares. First observe that Λ = ΛΛ, so Λ cannot be in the language. Consider $w_1 \neq w_2 $ and $ w_1w_2 \in L$. Let $w_3 = w_1w_2$, since $Λ \notin L$, $w_3 \neq w_2$, so $w_4 = w_2w_3 \in L$ where $w_4 \neq w_1$, finally let $w_5 = w_1w_4 \in L$. However, $w5 = w_1w_2w_1w_2 = w_3w_3$ which is square, so $w_5 \notin L$.

---
**Problem 2.14:** Let us define $(S^{**})^{*} = S^{***}$.

1. Is this set bigger than $S^{*}$?
2. Is it bigger that $S$?

**Answer 2.14:**

1. No. $(S^{**})^{*} = (S^{*})^{*} = S^{*}$ by Theorem 1.
2. It is often bigger than $S$.

---
**Problem 2.15:** Let $w$ be a string of letters and let the language $T$ be defined by adding $w$ to the language $S$. Suppose further that $T^{*} = S^{*}$.
* (i) Is it necessarily true that $w \in S$?
* (ii) Is it necessarily true that $w \in S^{*}$?

**Answer 2.15:**
* (i) no
* (ii) yes, $T = S + \{w\} \implies w \in T \implies w \in T^*$ and $T^* = S^* \implies w \in S^*$.

---
**Problem 2.16:** Give an example of a set $S$ such that the language $S^{*}$ has more six-letter words than eight-letter words. Does there exists an $S^{*}$ such that it has more six-letter words that twelve-letter words?

**Answer 2.16:**
Let $S = $ {*aaa*}, $S^*$ has one six-letter word and no seven-letter words and no eight-letter words. However, it is impossible for $S^*$ for $S$ to contain more six-letter words that twelve-letter words, because for every six-letter word $w$ there is a twelve-letter word $ww$ in $S^*$.

---
**Problem 2.17:**
* (i) Consider the language $S^{*}$, where $S$ = {*aa ab ba bb*}. Give another description of this language.
* (ii) Give an example of a set $S$ such that $S^{*}$ *only* contains all possible strings of *a*'s and *b*'s that have length divisible by 3.
* (iii) Let S be all strings of *a*'s and *b*'s with odd length. What is $S^{*}$?

**Answer 2.17:**
- (i) All words over $Σ =$ {*a b*} of even length.
- (ii) $S =$ {*aaa aab aba abb baa bab bba bbb*}
- (iii) All strings of *a*'s and *b*'s except Λ

---
**Problem 2.18:**
* (i) If $S =$ {*a b*} and $T^{*} = S^{*}$, prove that $T$ must contain $S$.
* (ii) Find another pair of sets $S$ and $T$ such that if $T^{*} = S^{*}$, then $S \subset T$.

**Answer 2.18:**
- (i) $S^*$ and $T^*$ both represent the set of all strings of *a*'s *b*'s. Therefore $T$ must include at least the words *a* and *b*, which is the set $S$.
- (ii) $S =$ {*a bb*}, $T =$ {*a aa bb*}.

---
**Problem 2.19:** One student suggested the following algorithm to test a string of *a*'s and *b*'s to see if it is a word in $S^{*}$, where $S =$ {*aa ba aba abaab*}.
* Step 1, cross off the longest set of characters from the front of the string that is a word in $S$.
* Step 2, repeat step 1 until it is no longer possible. If what remains is the string $Λ$, the original string was a word in $S^{*}$. If what remains is not $Λ$ (this means that some letters are left, but we cannot find a word in $S$ at the beginning), the original string was not a word in $S^{*}$. Find a string that disproves this algorithm.

**Answer 2.19:**
The word *abaaba* disproves the algorithm.

---
**Problem 2.20:** A language $L_1$ is smaller than another language $L_2$ if $L_1 \subset L_2$ and $L_1 \neq L_2$. Let $T$ be any language closed under concatenation; that is, if $t_1 \in T$ and $t_2 \in T$, then $t_1t_2$ is also an element of $T$. Show that if $T$ contains $S$ but $T \neq S^{*}$, then $S^{*}$ is smaller that $T$. We can summarize this by saying that $S^{*}$ is the smallest closed language containing $S$.

**Answer 2.20:**
Since $T$ is closed and $S \subset T$, any factors in $S$ concatentated together two at a time will be a word in $T$. Likewise concatentating factors in $S$ any number of times produces a word in $T$. That is any word in $S^*$ is also in $T$. However we are given that $T \neq S^*$ so $T$ contains some words not in $S^*$. We can conclude that $S^*$ is a proper subset of $T$, in other words $S^*$ is smaller that $T$, and in symbols $S^* \subset T$.

---
**Chapter 3: Recursive Definitions**

1. Write a recursive definition given a statement of the language
2. Prove a recursive definition correctly specifies the stated language
3. Show derivation chain (*x*) or tree(*x* and *y*) for instance of proof using recursive definition
4. Given a recursive definition of a language, prove a certain characteristic
5. Give a recursive definition, list the invalid substrings of length 2 or 3
6. Given a recursive definition, determine cardinalities based on combinations and permutations

---
**Problem 3.1:**
Write another recursive definition for the language $L_1$ of Chapter 2.

$L_1 = \{x\}^+$

**Answer 3.1:**

* Rule 1: $x$ is in $L_1$.
* Rule 2: If $w$ is in $L_1$, then so is $wx$.

---
**Problem 3.2:**
Using the second recursive definition of the set EVEN, how many different ways can we prove that 14 is in EVEN?
* Rule 1: 2 is in EVEN.
* Rule 2: If *x* and *y* are both in EVEN, then so is $x+y$.

**Answer 3.2:**

* 2+2=4
  * 2+4=6
    * 2+6=8
      * 2+8=10
        * 2+10=12
          * **2+12=14**
        * **4+10=14**
      * 4+8=12
        * **2+12=14**
      * **6+8=14**
    * 4+6=10
      * 2+10=12
        * **2+12=14**
      * **4+10=14**
    * 6+6=12
      * **2+12=14**
  * 4+4=8
    * 2+8=10
      * 2+10=12
        * **2+12=14**
      * **4+10=14**
    * 4+8=12
      * **2+12=14**
    * **6+8=14**

---
**Problem 3.3:**
Using the second recursive definition of EVEN.
* What is the smallest number of steps required to prove that 100 is EVEN?
* Describe a good method for showing that $2n$ is in EVEN.

**Answer 3.3:**
- 2, 2+2=4, 4+4=8, 8+8=16, 16+16=32, 32+32=64, 64+32=96, 96+4=100, eight steps
- Continue doubling until overreach. Back up one. Continue adding highest value in set which will not cause overreach. Repeat until target is reached.

---
**Problem 3.4:**
Show that the following is another recursive definition of the set EVEN:
* Rule 1: 2 and 4 are in EVEN
* Rule 2: If $x$ is in EVEN, then so is $x + 4$.

**Answer 3.4:**

We define the set $E$ recursively as follows:

1. **Base cases:** $2 \in E$ and $4 \in E$.
2. **Recursive step:** If $x \in E$, then $x + 4 \in E$.

**Step 1: Show $E \subseteq \text{EVEN}$.**

We proceed by structural induction on the definition of $E$:

- **Base cases:** $2$ and $4$ are even numbers.
- **Inductive step:** Assume $x$ is even. Then $x + 4$ is also even, since the sum of two even numbers is even.

Thus, every element of $E$ is even.

**Step 2: Show $\text{EVEN} \subseteq E$.**

Let $n$ be an even integer with $n \ge 2$.

We can write $n$ as either $n = 2 + 4k \quad \text{or} \quad n = 4 + 4k \quad$ for some integer $k \ge 0$.

- If $n = 2$ or $n = 4$, then $n \in E$ by the base cases.
- If $n > 4$, then $n - 4$ is even and $\ge 2$.  
  By the induction hypothesis, $n - 4 \in E$, and by the recursive step, $n \in E$.

**Step 3: Conclusion**

We have shown: $E \subseteq \text{EVEN} \quad \text{and} \quad \text{EVEN} \subseteq E$

Therefore: $E = \{ n \in \mathbb{N} \mid n \text{ is even and } n \ge 2 \}$.

---
**Problem 3.5:**
Show that there are infinitely many different recursive definitions for the set EVEN.

**Answer 3.5:**

$
\textbf{Theorem:} \quad
\text{There are infinitely many distinct recursive definitions of the set } \mathrm{EVEN}.
$

$
\textbf{Definition for a fixed } k \ge 1:
\quad
\begin{cases}
\text{Base cases:} & 2, 4, \dots, 2k \in E_k, \\
\text{Recursive step:} & x \in E_k \ \Rightarrow\ x + 2k \in E_k.
\end{cases}
$

$\textbf{Claim:} \quad E_k = \mathrm{EVEN} \quad \text{for all } k \ge 1.$

$\textit{Proof:}$

$\text{(1) Soundness: All base elements are even. Adding $2k$ preserves evenness, so } E_k \subseteq \mathrm{EVEN}.$

$
\text{(2) Completeness: Let $n \ge 2$ be even. Write $n = 2q$. By the division algorithm,}
$

$$q = kq' + j, \quad \text{with } q' \ge 0, \ j \in \{1,2,\dots,k\}.$$

$\text{Then } n = 2j + 2kq'. \ \text{Since $2j$ is a base element and we can add $2k$ repeatedly, } n \in E_k.$

$$\therefore \ E_k = \mathrm{EVEN} \ \text{for all } k \ge 1.$$

$$
\textbf{Infinitely many definitions:} \quad
\text{Different $k$ give different base sets and step sizes, so the rules differ.}
$$

$\text{Since $k$ can be any positive integer, there are infinitely many such recursive definitions.}$

---
**Problem 3.6:**
Using any recursive definition of the set EVEN, show that all the numbers in it end in the digits 0, 2, 4, 6, or 8.

**Answer 3.6:**

We define the set $\mathrm{EVEN}$ recursively as:

1. **Base case:** $2 \in \mathrm{EVEN}$.
2. **Recursive step:** If $x \in \mathrm{EVEN}$, then $x + 2 \in \mathrm{EVEN}$.
3. **Closure:** No other elements are in $\mathrm{EVEN}$.

**Claim:** Every $n \in \mathrm{EVEN}$ ends with digit $0, 2, 4, 6,$ or $8$.

**Proof (structural induction):**

**Base case:**  
$2$ ends with digit $2 \in \{0,2,4,6,8\}$.

**Inductive step:**  
Assume $x$ ends with digit $d \in \{0,2,4,6,8\}$.  
Consider $x + 2$:

$$
\begin{aligned}
d &= 0 \implies d+2 = 2, \\
d &= 2 \implies d+2 = 4, \\
d &= 4 \implies d+2 = 6, \\
d &= 6 \implies d+2 = 8, \\
d &= 8 \implies d+2 = 10 \ (\text{last digit } 0).
\end{aligned}
$$

In each case, the last digit remains in $\{0,2,4,6,8\}$.

**Conclusion:**  
By induction, all elements of $\mathrm{EVEN}$ have last digit in $\{0,2,4,6,8\}$.

---
**Problem 3.7:**
The set POLYNOMIAL defined in this chapter contains only the polynomials in the one variable $x$. Write a recursive definition for the set of all polynomials in the two variables $x$ and $y$.

* Rule 1: Any number is in POLYNOMIAL.
* Rule 2: The variable *x* is in POLYNOMIAL.
* Rule 3: If *p* and *q* are in POLYNOMIAL, then so are $p+q$, $p-q$, $(p)$, and $pq$.

**Answer 3.7:**

- Rule 1: Any number is in POLYNOMIAL.
- Rule 2: The variable *x* or *y* is in POLYNOMIAL.
- Rule 3: If *p* and *q* are in POLYNOMIAL, then so are $p+q$, $p-q$, $(p)$, and $pq$.

---
**Problem 3.8:**
Define the set of valid algebraic expressions ALEX as follows:
* Rule 1: All polynomials are in ALEX.
* Rule 2: If $f(x)$ and $g(x)$ are in ALEX, then so are:
* (i) $(f(x))$
* (ii) $-(f(x))$
* (iii) $f(x) + g(x)$
* (iv) $f(x) - g(x)$
* (v) $f(x)g(x)$
* (vi) $f(x)/g(x)$
* (vii) $f(x)^{g(x)}$
* (viii) $f(g(x))$

- (a) Show that $(x + 2)^{3x}$ is in ALEX.
- (b) Show that elementary calculus contains enough rules to prove the theorem that all algebraic expressions can be differentiated.
- (c) Is Rule 2(viii) really necessary?

**Answer 3.8:**

- (a) Steps for proof:
  - *Rule 1.2:* $x$
  - *Rule 1.1:* $2$
  - *Rule 1.1:* $3$
  - *Rule 1.3:* $x+2$
  - *Rule 1.3:* $(x+2)$
  - *Rule 1.3:* $3x$
  - *Rule 2:* $(x+2)^{3x}$
- (b) Rules of differentiation exists for all unary and binary operators of POLYNOMIAL.
- (c) No.

---
**Problem 3.9:**
* Using the fact that $3x^2 + 7x - 9 = (((((3)x) + 7)x) - 9)$, show how to produce this polynomial from the rules for POLYNOMIAL using multiplication only twice.
* What is the smallest number of steps needed for producing $x^8 + x^4$?
* What is the smallest number of steps needed for producing $7x^7 + 5x^5 + 3x^3 + x$?

**Answer 3.9:**
- The product of $(3)$ and $x$, and the product of $(((3)x) + 7)$ and $x$.
- Steps: $x$, $8$, $4$, $x^8$, $x^4$, $x^8 + x^4$, six steps.
- Steps:
  - 1: $7$
  - 2: $x$
  - 3: $5$
  - 4: $3$
  - 5: $x^7$
  - 6: $x^5$
  - 7: $x^3$
  - 8: $7x^7 + 5x^5$
  - 9: $7x^7 + 5x^5 + 3x^3$
  - 10: $7x^7 + 5x^5 + 3x^3 + x$

---
**Problem 3.10:**
Show that if $n$ is less that 31, then $x^n$ can be shown to be in POLYNOMIAL in fewer that eight steps.

**Answer 3.10:**

---
**Problem 3.11:**
In this chapter, we mentioned several substrings of length 2 that cannot occur in arithmetic expressions, such as *(/*, *+)*, *//*, and *\*/*. What is the complete list of substrings of length 2 that cannot occur?

**Answer 3.11:**
- *++* *-+* *\*+* */+* *(+*
- *)(* *+\** *-\** */\** *(\**
- *+/* *-/* *\*/* *//* *(/*
- *+)* *-)* *\*)* */)* *()*

---
**Problem 3.12:**
Are there any substrings of length 3 that cannot occur that do not contain forbidden substrings of length 2? (This means that *///* is already known to be illegal because it contains the forbidden substring *//*.) What is the longest forbidden substring that does not contain a shorter forbidden substring?

**Answer 3.12:**
Yes. *+--* *---* *\*--* */--* *(--* *\*\*\**

---
**Problem 3.13:**
The rules given earlier for the set AE allow for the peculiar expressions

$(((((9)))))$ and $-(-(-(-(9))))$

It is not really harmful to allow these in AE, but is there some modified definition of AE that eliminates this problem?

**Answer 3.13:** Yes, but with some difficulty. Though, it's almost always better to remove the redunancies and equivalencies in a later processing step.

---
**Problem 3.14:**
* (i) Write out the full recursive definition for the propositional calculus that contains the symbols $\vee$ and $\wedge$ as well as $¬$ and $→$.
* (ii) What are all the forbidden substrings of length 2 in this language?

**Answer 3.14:**
- (i) PROPCALC is defined as
  - Rule 1: Any letter is in PROPCALC.
  - Rule 2: If *x* and *y* are in PROPCALC, then so are the following:
    - $¬x$
    - $(x)$
    - $x\vee y$
    - $x\wedge y$
    - $x→y$

- (ii) 22 forbidden substrings of length 2
  - $)($, $)¬$
  - $()$, $(\vee$, $(\wedge$, $(→$
  - $¬)$, $¬\vee$, $¬\wedge$, $¬→$
  - $\vee)$, $\vee\vee$, $\vee\wedge$, $\vee →$
  - $\wedge)$, $\wedge\vee$, $\wedge\wedge$, $\wedge →$
  - $→)$, $→\vee$, $→\wedge$, $→→$

---
**Problem 3.15:**
* (i) When asked to give a recursive definition for the language PALINDROME over the alphabet $Σ = \{a\, b\}$, a student wrote:
  * Rule 1: *a* and *b* are in PALINDROME
  * Rule 2: If *x* is in PALINDROME, then so are *axa* and *bxb*.

Unfortunately, all the words in the language defined above have an odd length and so it is not all of PALINDROME.
* (i)  Fix this problem.
* (ii) Give a recursive definition for the language EVEN_PALINDROME of all palindromes of even length.

**Answer 3.15:**

- (i) PALINDROME is defined as
  - Rule 1: *a*, *b*, and $Λ$ are in PALINDROME
  - Rule 2: If *x* is in PALINDROME, then so are $xx$, $axa$, and $bxb$.

- (ii) EVEN_PALINDROME is define as
  - Rule 1: *aa*, *bb* are in EVEN_PALINDROME
  - Rule 2: If *x* is in EVEN_PALINDROME, then so are $axa$ and $bxb$.

---
**Problem 3.16:**
* (i) Give a recursive definition for the set $ODD = \{1\, 3\, 5\, 7\, ...\}$.
* (ii) Give a recursive definition for the set of strings of digits 0, 1, 2, 3, ..., 9 that cannot start with the digit 0.

**Answer 3.16:**
- $ODD$ is defined as
  - Rule 1: 1 is in $ODD$
  - Rule 2: If *x* is in $ODD$, then so is $x + 2$.

- $DIGITS$ is defined as
  - Rule 1: $\{1\,2\,3\,4\,5\,6\,7\,8\,9\} \subset DIGITS$.
  - Rule 2: If $x$ and $y$ are in $DIGITS$, then so is $x0$ and $xy$.

---
**Problem 3.17:**
In this chapter we attempted to define the positive numbers by the following rules:
* Rule 1: 1 is in $L$
* Rule 2: If $x$ and $y$ are in $L$, Then so are $x+y$, $x*y$, and $x/y$.

The language $L$ defined in this way is a famous mathematical set.
* (i) What is it?
* (ii) Prove it.

**Answer 3.17:**
- (i) The set of positive rational numbers.
- (ii) The additional operator gives the positive natural numbers. The division operators gives the fractional rational numbers.

---
**Problem 3.18:**
Give two recursive definitions for the set POWERS-OF-TWO $= \{1\, 2\, 4\, 8\, 16\, ...\}$.

Use one of them to prove that the product of two POWERS-OF-TWO is also a POWERS-OF-TWO.

**Answer 3.18:**

- $POWERS\_OF\_TWO$ is defined as
  - Rule 1: $1 \in POWERS\_OF\_TWO$
  - Rule 2: $x \in POWERS\_OF\_TWO \implies 2x \in POWERS\_OF\_TWO$

Claim: $x, y \in POWERS\_OF\_TWO \implies x*y \in POWERS\_OF\_TWO$

Proof:

Given $x, y \in POWERS\_OF\_TWO$.

Recall $x=2^m$ and $y=2^n$, for some m, n.

So $x*y = 2^m2^n = 2^{m+n}$, a power of 2.

$\therefore x*y \in POWERS\_OF\_TWO$.

---
**Problem 3.19:**
Give recursive definitions for the the following languages over the alphabet $\{a\, b\}$:
* (i) The language EVENSTRING of all words of even length.
* (ii) The language ODDSTRING of all words of odd length.
* (iii) The language AA of all words containing the substring *aa*.
* (iv)  The language NOTAA of all words not containing the substring *aa*.

**Answer 3.19:**
- (i) $EVENSTRING$ is defined as
  - Rule 1: $Λ \in EVENSTRING.$
  - Rule 2: $w \in EVENSTRING \implies waa$, $wab$, $wba$, $wbb \in EVENSTRING$.
- (ii) $ODDSTRING$ is defined as
  - Rule 1: $a, b \in ODDSTRING$.
  - Rule 2: $w \in ODDSTRING \implies waa$, $wab$, $wba$, $wbb \in ODDSTRING$.
- (iii) $AA$ is defined as
  - Rule 1: $aa \in AA$.
  - Rule 2: $w \in AA \implies wa, aw, wb, bw \in AA$.
- (iv) $NOTAA$ is defined as
  - Rule 1: $Λ, a, b \in NOTAA$.
  - Rule 2: $w \in NOTAA \implies wb, wba \in NOTAA$.

---
**Problem 3.20:**
* (i) Consider the following recursive definition of 3-PERMUTATION:
  * Rule 1: 123 is a 3-PERMUTATION.
  * Rule 2: If *xyz* is a 3-PERMUTATION, then so are *zyx* and *yzx*.

  Show that there are six different 3-PERMUTATIONs.

* (ii)
  * Rule 1: 1234 is a 4-PERMUTATION.
  * Rule 2: If *xyzw* is a 4-PERMUTATION, then so are *wzyx* and *yzwx*.

  How many 4-PERMUTATIONs are there (by this definition)?

**Answer 3.20:**
- (i) 123, 321, 231, 213, 312, 132
- (ii) 1234, 4321, 2341, 3214, 1432, 3412, 4123, 2143

In [ ]:
def derivations(n):
    if n == 1:
        return ["1"]
    results = []
    for a in range(1, n):
        b = n - a
        if a > b:
            break
        for left in derivations(a):
            for right in derivations(b):
                if a == b:
                    if left > right:
                        continue
                results.append(f"{a}[{left}] + {b}[{right}]")
    return results

n = 7
seq = derivations(n)
print(f"Derivations for {n} (total {len(seq)} ways):")
for i, d in enumerate(seq, 1):
    print(f"{i}. {d}")

In [ ]:
from functools import lru_cache

@lru_cache(None)
def count_proofs(n):
    if n == 1:
        return 1
    total = 0
    for a in range(1, n):
        b = n - a
        if a > b:
            break
        if a == b:
            total += (count_proofs(a) * (count_proofs(a) + 1)) // 2
        else:
            total += count_proofs(a) * count_proofs(b)
    return total

for n in range(1, 16):
    print(f"{n}: {count_proofs(n)} proofs")

---
**Chapter 4: Regular Expressions**

---
**Problem 4.1:**
Let $r_1$, $r_2$, and $r_3$ be three regular expressions.
Show that the language associated with $(r_1+r_2)r_3$ is the same as the language associated with $r_1r_3+r_2r_3$.
Show that $r_1(r_2+r_3)$ is equivalent to $r_1r_2+r_1r_3$.
This will be the same as proving a "distributive law" for regular expressions.

**Answer 4.1:**

Let $L(r)$ be the language denoted by the regular expression $r$.

- Union: $L(r_1 + r_2) = L(r_1) \cup L(r_2)$
- Concatenation: $L(r_1 r_2) = \{\, xy \mid x \in L(r_1),\ y \in L(r_2) \,\}$

Claim: $$L\big((r_1 + r_2) r_3\big) = L(r_1 r_3) \cup L(r_2 r_3).$$

Proof:
- ($\subseteq$) Let $w \in L((r_1 + r_2) r_3)$.
  - Then $w = xy$ with $x \in L(r_1) \cup L(r_2)$ and $y \in L(r_3)$.
  - If $x \in L(r_1)$, then $w \in L(r_1 r_3)$.
  - If $x \in L(r_2)$, then $w \in L(r_2 r_3)$.
  - Hence $w \in L(r_1 r_3) \cup L(r_2 r_3)$.

- ($\supseteq$) Let $w \in L(r_1 r_3) \cup L(r_2 r_3)$.
  - If $w \in L(r_1 r_3)$, then $w = xy$ with $x \in L(r_1)$ and $y \in L(r_3)$.
  - Since $x \in L(r_1) \subseteq L(r_1) \cup L(r_2)$, we have $w \in L((r_1 + r_2) r_3)$.
  - The case $w \in L(r_2 r_3)$ is analogous.

Therefore, $$L((r_1 + r_2) r_3) = L(r_1 r_3) \cup L(r_2 r_3).$$

Claim: $$L\big(r_1 (r_2 + r_3)\big) = L(r_1 r_2) \cup L(r_1 r_3).$$

Proof:
- ($\subseteq$) Let $w \in L(r_1 (r_2 + r_3))$.
  - Then $w = xy$ with $x \in L(r_1)$ and $y \in L(r_2) \cup L(r_3)$.
  - If $y \in L(r_2)$, then $w \in L(r_1 r_2)$.
  - If $y \in L(r_3)$, then $w \in L(r_1 r_3)$.
  - Hence $w \in L(r_1 r_2) \cup L(r_1 r_3)$.

- ($\supseteq$) Let $w \in L(r_1 r_2) \cup L(r_1 r_3)$.
  - If $w \in L(r_1 r_2)$, then $w = xy$ with $x \in L(r_1)$ and $y \in L(r_2) \subseteq L(r_2) \cup L(r_3)$.
  - Thus $w \in L(r_1 (r_2 + r_3))$.
  - The case $w \in L(r_1 r_3)$ is analogous.

Therefore, $$L(r_1 (r_2 + r_3)) = L(r_1 r_2) \cup L(r_1 r_3).$$

Conclusion:
$$
(r_1 + r_2) r_3 \equiv r_1 r_3 + r_2 r_3
\quad\text{and}\quad
r_1 (r_2 + r_3) \equiv r_1 r_2 + r_1 r_3.
$$

For Problems 4.2 through 4.11, construct a regular expression defining each of the following languages over the alphabet $Σ = \{a\,b\}$.

A recursive definition for REGEX is:
* Rule 1: $Λ, a, b \in REGEX$.
* Rule 2: If $r_1, r_2 \in REGEX$, then $(r_1), r_1r_2, r_1+r_2, r_1^{*} \in REGEX$.

---
**Problem 4.2:** Given $Σ = \{a\,b\}$. Construct a regular expression for all words in which *a* appears tripled, if at all. This means that every clump of *a*'s contains 3 or 6 or 9 or 12 ... *a*'s.

**Answer 4.2:**
- $(aaa+b)^{*}$
- $((aaa)^{*}b^{*})^{*}$

---
**Problem 4.3:** Given $Σ = \{a\,b\}$. Construct a regular expression for all words that contain at least one of the strings $s_1$, $s_2$, $s_3$, or $s_4$.

**Answer 4.3:** $(a+b)^{*}(s_1+s_2+s_3+s_4)(a+b)^{*}$

---
**Problem 4.4:** Given $Σ = \{a\,b\}$. Construct a regular expression for all words that contain exactly two *b*'s or exactly three *b*'s, not more.

**Answer 4.4:** $a^{*}ba^{*}ba^{*}(Λ+ba^{*})$

---
**Problem 4.5:** Given $Σ = \{a\,b\}$. Construct a regular expression for
* (i) all strings that end in a double letter,
* (ii) all strings that do not end in a double letter.

**Answer 4.5:**
- $(a+b)^{*}(aa+bb)$
- $Λ+a+b+(a+b)^{*}(ab+ba)$

---
**Problem 4.6:** Given $Σ = \{a\,b\}$. Construct a regular expression for all strings that have exactly one double letter in them.

**Answer 4.6:** $(ab)^{*}aa(ba)^{*} + (ba)^{*}bb(ab)^{*}$

---
**Problem 4.7:** Given $Σ = \{a\,b\}$. Construct a regular expression for all strings in which the letter *b* is *never* tripled. This means that no word contains the string *bbb*.

**Answer 4.7:** $(Λ+b+bb)(a^{*}a(b+bb))^{*}a^{*}$

---
**Problem 4.8:** Given $Σ = \{a\,b\}$. Construct a regular expression for all words in which *a* is tripled or *b* is tripled, but not both. This means each word contains the substring *aaa* or the substring *bbb* but not both.

**Answer 4.8:**
- Let $NOTBBB=(Λ+b+bb)(a^{*}a(b+bb))^{*}a^{*}$
- Let $HASAAA=(NOTBBB)aaa(NOTBBB)$
- Let $NOTAAA=(Λ+a+aa)(b^{*}b(a+aa))^{*}b^{*}$
- Let $HASBBB=(NOTBBB)bbb(NOTBBB)$
- $(HASAAA+HASBBB)$

---
**Problem 4.9:** Given $Σ = \{a\,b\}$. Construct a regular expression for
* (i) all words that do not have the substring *ab*,
* (ii) all words that do not have both the substrings *bba* and *abb*.

**Answer 4.9:**
- (i) $b^{*}a^{*}$
- (ii) $Λ+a^{*}b(Λ+a)+b^{*}a(Λ+b)$

---
**Problem 4.10:** Given $Σ = \{a\,b\}$. Construct a regular expression for all strings in which the *total* number of *a*'s is divisible by 3 no matter how they are distributed, such as *aabaabbaba*.

**Answer 4.10:** $b^{*}(ab^{*}ab^{*}a)^{*}b^{*}$

---
**Problem 4.11:** Given $Σ = \{a\,b\}$. Construct a regular expression for all strings
* (i) in which any *b*'s that occur are found in clumps of an odd number at a time, such as *abaabbbab*.
* (ii) that have an even number of *a*'s and an odd number of *b*'s.
* (iii) that have an odd number of *a*'s and an odd number of *b*'s.

**Answer 4.11:**

- Let $EVEN\_EVEN=(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}$
- (i) $b(bb)^{*}(ab(bb)^{*})^{*}a^{*}$
- (ii)
  - Let $ODD=b+ab(bb)^{*}a$
  - $(EVEN\_EVEN)(ODD)(EVEN\_EVEN)$
- (iii)
  - Let $ODD=ab+ba$
  - $(EVEN\_EVEN)(ODD)(EVEN\_EVEN)$

---
**Problem 4.12:**

* (i) Let us reconsider the regular expression $$(a+b)^{*}a(a+b)^{*}b(a+b)^{*}$$ Show that this is equivalent to $$(a+b)^{*}ab(a+b)^{*}$$ in the sense that they define the same language.
* (ii) Show that $(a+b)^{*}ab(a+b)^{*} + b^{*}a^{*} = (a+b)^{*}$.
* (iii) Show that $(a+b)^{*}ab(a+b)^{*}ab(a+b)^{*} = (a+b)^{*}$.
* (iv) Is (iii) the last variation of this theme or are there more beasts left in this cave?

**Answer 4.12(i):**

Let $L_1 = L\big((a+b)^{*}a(a+b)^{*}b(a+b)^{*}\big).$

Let $L_2 = L\big((a+b)^{*}ab(a+b)^{*}\big).$

Take any $w \in L_1$. Then $w$ can be written as  
$$
w = x \; a \; m \; b \; y
\quad\text{with}\quad
x,y \in \{a,b\}^{*}, \; m \in \{a,b\}^{*}.
$$

We analyze the possible forms of $m$:

- **Case 1: $m = \epsilon$**  
  Then $w = x\,ab\,y$, so $w$ contains the substring $ab$ and hence $w \in L_2$.

- **Case 2: $m \neq \epsilon$ and $m$ contains a $b$**  
  Let the first $b$ in $m$ occur at position $j$.  
  - If $j = 1$, then the boundary between $a$ and the first symbol of $m$ is $ab$, so $w$ contains $ab$.  
  - If $j > 1$, then the symbol at position $j-1$ in $m$ cannot be $b$ (by minimality of $j$), so it is $a$, giving an internal $ab$ in $m$.  
  In both subcases, $w \in L_2$.

- **Case 3: $m$ has no $b$**  
  Then $m = a^{k}$ for some $k \ge 1$. The last $a$ of $m$ is immediately followed by the trailing $b$, so $ab$ occurs in $w$. Thus $w \in L_2$.

In all cases, $w$ contains a contiguous $ab$, so $w \in L_2$.

If $w \in L_2$, then  
$$
w = x\,ab\,y, \quad x,y \in \{a,b\}^{*}.
$$
This matches $(a+b)^{*}a(a+b)^{*}b(a+b)^{*}$ by choosing $m = \epsilon$.  
Hence $w \in L_1$.

Since $L_1 \subseteq L_2$ and $L_2 \subseteq L_1$, we have  
$$
L\big((a+b)^{*}a(a+b)^{*}b(a+b)^{*}\big)
=
L\big((a+b)^{*}ab(a+b)^{*}\big).
$$

**Answer 4.12(ii):**

Since $(a+b)^{*}ab(a+b)^{*}ab(a+b)^{*}$ represents all strings which have a *b* following an *a*. This exludes from the set of all possible strings, $a^{*}$, $b^{*}$, and $b^{*}a^{*}$. Hence the union of the two is the set of all possible string $(a+b)^{*}$.

**Answer 4.12(iii):**

Strictly speaking there are an infinite amount of these type of substitutions.

---
**Problem 4.13:**

We have defined the product of two sets of strings in general. If we apply this to the case where both factors are the same set, $S=T$, we obtain squares, $S^2$. Similarily, we can define $S^3, S^4,...$. Show that it makes some sense to write:
* (i) $S^{*} = Λ + S + S^1 + S^2 + S^3 + S^4 + ...$
* (ii) $S^{+} = S + S^1 + S^2 + S^3 + S^4 + ...$

**Answer 4.13:**

- (i) Obvious.
- (ii) Obvious.

---
**Problem 4.14:** If the only difference between $L$ and $L^{*}$ is the word $Λ$, is the only difference between $L^2$ and $L^{*}$ the word $Λ$?

**Answer 4.14:**

Let $L=(a+b)^{+}$. $L$ and $L^{*}$ differ only by $Λ$.

Now $L^2=(a+b)(a+b)^{+}$. So, no. $L$ and $L^{*}$ differ by $Λ$, $a$, and $b$.

---
**Problem 4.15:** Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $(ab)^{*}a\quad$ and $\quad a(ba)^{*}$
* (ii) $(a^{*}+b)^{*}\quad$ and $\quad (a+b)^{*}$
* (iii) $(a^{*}+b^{*})^{*}\quad$ and $\quad (a+b)^{*}$

**Answer 4.15(i):**

Let the alphabet be $\Sigma = \{a,b\}$ with concatenation associative and identity $\epsilon$.

Define  
- $w_0 = \epsilon$, $w_{k+1} = w_k\,ab$ so $w_k = (ab)^k$,  
- $v_0 = \epsilon$, $v_{k+1} = v_k\,ba$ so $v_k = (ba)^k$.

**Claim:** for all $k \ge 0$, $w_k\,a = a\,v_k$.

**Base case ($k=0$):** $\epsilon a = a = a\epsilon.$

**Inductive step:**

Assume $w_k a = a v_k$.

Then  
$$
\begin{aligned}
w_{k+1} a
&= (w_k\,ab)\,a \\
&= w_k\,(ab\,a) && \text{(associativity)} \\
&= (w_k a)\,ba \\
&= (a v_k)\,ba && \text{(inductive hypothesis)} \\
&= a\,(v_k\,ba) \\
&= a\,v_{k+1}.
\end{aligned}
$$

Thus $w_k a = a v_k$ for all $k$, i.e., $(ab)^k a = a(ba)^k$.

**Answer 4.15(ii)**

We work over the alphabet $\Sigma = \{a, b\}$. The claim is  
$$
(a^{*} + b)^{*} \;=\; \Sigma^{*} \;=\; (a + b)^{*}.
$$

**Claim:** Every $w \in \Sigma^{*}$ belongs to $(a^{*} + b)^{*}$.

**Base case: ($|w| = 0$):**  
$$
w = \epsilon \in (a^{*} + b)^{*}.
$$

**Inductive step:** Assume all words of length $\le n$ are in $(a^{*} + b)^{*}$. Take $w$ with $|w| = n+1$.

- **Case 1:** $w$ starts with $b$. Write $w = b\,u$. By the inductive hypothesis, $u \in (a^{*} + b)^{*}$, hence $w \in (a^{*} + b)^{*}$ since $b \in a^{*} + b$.

- **Case 2:** $w$ starts with $a$. Let $w = a^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $b$. Then $a^{k} \in a^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b)^{*}$. Hence $w \in (a^{*} + b)^{*}$.

Thus  
$$
\Sigma^{*} \subseteq (a^{*} + b)^{*},
$$
and the equality with $(a + b)^{*}$ follows as above.

**Answer 4.15(iii)**

We work over the alphabet $\Sigma = \{a, b\}$. The claim is  
$$
(a^{*} + b^{*})^{*} \;=\; \Sigma^{*} \;=\; (a + b)^{*}.
$$

**Claim:** Every $w \in \Sigma^{*}$ belongs to $(a^{*} + b^{*})^{*}$.

**Base case ($|w| = 0$):**  
$$
w = \epsilon \in (a^{*} + b^{*})^{*}.
$$

**Inductive step:** Assume all words of length $\le n$ are in $(a^{*} + b^{*})^{*}$. Take $w$ with $|w| = n+1$.

- **Case 1:** $w$ starts with $b$. Let $w = b^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $a$. Then $b^{k} \in b^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b^{*})^{*}$. Hence $w \in (a^{*} + b^{*})^{*}$.

- **Case 2:** $w$ starts with $a$. Let $w = a^{k} u$ where $k \ge 1$ and $u$ is empty or starts with $b$. Then $a^{k} \in a^{*}$ and, by the inductive hypothesis, $u \in (a^{*} + b^{*})^{*}$. Hence $w \in (a^{*} + b^{*})^{*}$.

Thus  
$$
\Sigma^{*} \subseteq (a^{*} + b^{*})^{*},
$$
and the equality with $(a + b)^{*}$ follows as above.

---
**Problem 4.16:** Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $Λ^{*}\quad$ and $\quad Λ$
* (ii) $(a^{*}b)^{*}a^{*}\quad$ and $\quad a^{*}(ba^{*})^{*}$
* (iii) $(a^{*}bbb)^{*}a^{*}\quad$ and $\quad a^{*}(bbba^{*})^{*}$

**Answer 4.16:**

- (i) Obvious.
- (ii) Follows from $(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$.
- (iii) Follows from $(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$.

---
**Problem 4.17:** Show that the following pairs of regular expressions define the same language over the alphabet $Σ = \{a\,b\}$:

* (i) $((a+bb)^{*}aa)^{*}\quad$ and $\quad Λ+(a+bb)^{*}aa$
* (ii) $(aa)^{*}(Λ+a)\quad$ and $\quad a^{*}$
* (iii) $a(aa)^{*}(Λ+a)b+b\quad$ and $\quad a^{*}b$
* (iv) $a(ba+a)^{*}b\quad$ and $\quad aa^{*}b(aa^{*}b)^{*}$
* (v) $Λ+a(a+b)^{*}+(a+b)^{*}aa(a+b)^{*}\quad$ and $\quad ((b^{*}a)^{*}ab^{*})^{*}$

**Answer 4.17:**

- (i) Follows from definition of Kleene closure.
- (ii) A combination of mutiple of two *a*'s followed by an optional *a*, is simply zero or more *a*'s.
- (iii) Follows from (ii) above.
- (iv) Regarding the first, strings begin with *a* and end with *b* and between *b*'s are always followed by an *a*. Or in other words, each *b* is surrounded by at least one *a* on either side.
- (v) $???$

---
**Problem 4.18:** Describe (in English phrases) the languages associated with the following regular expressions:

* (i) $(a+b)^{*}a(Λ+bbbb)$
* (ii) $(a(a+bb)^{*})^{*}$
* (iii) $(a(aa)^{*}b(bb)^{*})^{*}$
* (iv) $(b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}$
* (v) $(b(bb)^{*})^{*}(a(aa)^{*}b(bb)^{*})^{*}(a(aa)^{*})^{*}$
* (vi) $((a+b)a)^{*}$

**Answer 4.18:**

- all strings ending in either *a* or *abbbb*.
- all strings starting with *a* and in which *b*'s appear as clumps of even length.
- Λ or strings starting with a and ending in b where *a*'s and *b*'s are clumped by odd-numbered lengths.
- Λ or clumps of odd-length *a*'s and *b*'s ending in *b*.
- Λ or clumps of odd-length *a*'s and *b*'s.
- Λ or even-length strings with *a* repeating in even positions.

---
**Problem 4.19:** (D. N. Arden)

Let $R$, $S$, and $T$ be three languages and assume that $Λ$ is not in $S$.

Prove the following statements:
* From the premise that $R=SR+T$, we can conclude that $R=S^{*}T$
* From the premise that $R=S^{*}T$, we can conclude that $R=SR+T$.

**Answer 4.19:**

$R=SR+T$ defines $R$ as containing all the words in $T$ and also all words composed of an initial part from $S$ concatenated with another words from $R$. This permits us to continue concatenating factors from $S$ as many time as we like, but we are still required to finish the word with a factor from $R$. The only type of factor from $R$ that does not entail this kind of recursion is a factor from the language $T$. So we have any number of factors from S followed by a factor from $T$, or we have just a factor from $T$, equivalently $S^{*}T$.

$R=S^{*}T$ defines $R$ as words with any number (if any) factors from $S$ followed by a single factor from $T$. If the star operator is taken fro zero, we have for $R$ any word in $T$. Suppose the star operator is taken once, then we have exactly one word from $S$ followed by one word from $T$. But the word from $T$ is itself a factor from $R$, so we can see that this word is in the set $SR$. Consider what happens if we take the star more than once; the star operates on $S$, so we have $SR, SSR, SSSR, ...$ all in the set $S^{*}T$. In fact, we established $S^{+}T$ in this way. The difference between $S^{*}T$ and $S^{+}T$ is that only $S^{*}T$ contains $T$. But that means that $S^{*}T$ must equal exactly $SR+T$.

**Answer 4.19:**

**Statement and assumptions**

Let $R, S, T \subseteq \Sigma^{*}$ be languages, and assume $\varepsilon \notin S$. Prove:

1. If $R = S R \cup T$, then $R = S^{*} T$.
2. If $R = S^{*} T$, then $R = S R \cup T$.

**Preliminaries**

- **Kleene star:** $S^{*} = \bigcup_{n \ge 0} S^{n}$ and $S^{*} = \{\varepsilon\} \cup S S^{*}$.
- **Length fact (uses $\varepsilon \notin S$):** Every $x \in S$ satisfies $|x| \ge 1$. Hence if $w = xy$ with $x \in S$, then $|y| < |w|$.

From $R = S R \cup T$ to $R = S^{*} T$

We prove both inclusions.

Inclusion $R \subseteq S^{*} T$:

Fix $w \in R$. Proceed by induction on $|w|$.

- **Base case:** If $w \in T$, then $w \in S^{0} T \subseteq S^{*} T$.
- **Inductive step:** If $w \notin T$, then $w \in S R$ by $R = S R \cup T$.  
  So $w = xy$ for some $x \in S$ and $y \in R$.  
  Since $\varepsilon \notin S$, we have $|x| \ge 1$, hence $|y| < |w|$.  
  By the induction hypothesis, $y \in S^{*} T$. Thus  
  $$w = x y \in S S^{*} T \subseteq S^{*} T.$$

Therefore $R \subseteq S^{*} T$.

Inclusion $S^{*} T \subseteq R$:

We show $S^{n} T \subseteq R$ for all $n \ge 0$ by induction on $n$.

- **Base case: ($n=0$):** $S^{0} T = T \subseteq R$ since $R = S R \cup T$.
- **Inductive step:** Suppose $S^{n} T \subseteq R$. Then  
  $$S^{n+1} T = S (S^{n} T) \subseteq S R \subseteq S R \cup T = R.$$

Hence $S^{*} T = \bigcup_{n \ge 0} S^{n} T \subseteq R$.

Combining both inclusions, we have $R = S^{*} T$.

From $R = S^{*} T$ to $R = S R \cup T$:

Using $S^{*} = \{\varepsilon\} \cup S S^{*}$, we have:
$$
R = S^{*} T
  = (\{\varepsilon\} \cup S S^{*}) T
  = T \cup S (S^{*} T)
  = T \cup S R
  = S R \cup T.
$$

This implication does not require $\varepsilon \notin S$.

**Conclusion**

Under $\varepsilon \notin S$, the equation $R = S R \cup T$ holds if and only if $R = S^{*} T$ (Arden’s lemma).

---
**Problem 4.20(i):**

Explain why we can take any pair of equivalent regular expressions and replace the letter *a* in both with any regular expression **R** and the letter *b* with any regular expression **S** and the resulting regular expressions will have the same language. For example, 17(ii), which says

   $$(a^{*}b)^{*}a^{*}=a^{*}(ba^{*})^{*}$$

becomes the identity

   $$(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$$

which is true for all regular expressions **R** and **S**.

**Answer 4.20(i):**

It is easiest to understand this in terms of the recursive definition of regular expressions. Any letter of the alphabet is itself a regular expression. The rule for forming regular expressions apply to these and to all more complicated regular expressions. Each regular expression describes a set. Equivalent expressions describe equivalent sets. The equivalences are not disturbed by taking unions, products, or Kleen closures. This means that any set must be considered a building block in the construction of other sets.

**Answer 4.20(i):**

**Substitution Preserves Equivalence of Regular Expressions**

**1. Setup**
Let $E_1$ and $E_2$ be regular expressions over the alphabet $\{a,b\}$ with  
$$
L(E_1) = L(E_2).
$$

Let $R$ and $S$ be arbitrary regular expressions (over any alphabet).  
We write  
$$
E[a \leftarrow R,\, b \leftarrow S]
$$  
for the expression obtained by replacing each $a$ by $R$ and each $b$ by $S$.

**2. Substitution on Words and Languages**
Define a substitution $\sigma$ on letters by:
$$
\sigma(a) = L(R), \quad \sigma(b) = L(S).
$$

Extend $\sigma$ to words $w = w_1 w_2 \dots w_n \in \{a,b\}^*$ by:
$$
\hat{\sigma}(w) = \sigma(w_1) \cdot \sigma(w_2) \cdots \sigma(w_n),
$$
and to languages $L \subseteq \{a,b\}^*$ by:
$$
\hat{\sigma}(L) = \bigcup_{w \in L} \hat{\sigma}(w).
$$

**3. Lemma — Semantics Commute with Substitution**
For every regular expression $E$ over $\{a,b\}$:
$$
L\big(E[a \leftarrow R,\, b \leftarrow S]\big) = \hat{\sigma}\big(L(E)\big).
$$

**Proof (by structural induction on $E$):**

1. **Empty set**:  
   $E = \emptyset$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = \emptyset = \hat{\sigma}(\emptyset).
   $$

2. **Empty word**:  
   $E = \varepsilon$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = \{\varepsilon\} = \hat{\sigma}(\{\varepsilon\}).
   $$

3. **Letter $a$**:  
   $E = a$  
   $$
   L(E[a \leftarrow R, b \leftarrow S]) = L(R) = \sigma(a) = \hat{\sigma}(\{a\}).
   $$

4. **Letter $b$**: symmetric to $a$.

5. **Union**: $E = F + G$  
   $$
   L\big((F+G)[\cdot]\big) = L(F[\cdot]) \cup L(G[\cdot])  
   = \hat{\sigma}(L(F)) \cup \hat{\sigma}(L(G))  
   = \hat{\sigma}(L(F) \cup L(G)).
   $$

6. **Concatenation**: $E = FG$  
   $$
   L\big((FG)[\cdot]\big) = L(F[\cdot]) \cdot L(G[\cdot])  
   = \hat{\sigma}(L(F)) \cdot \hat{\sigma}(L(G))  
   = \hat{\sigma}(L(F) \cdot L(G)).
   $$

7. **Kleene star**: $E = F^*$  
   $$
   L\big((F^*)[\cdot]\big) = \big(L(F[\cdot])\big)^*  
   = \big(\hat{\sigma}(L(F))\big)^*  
   = \hat{\sigma}\left(\bigcup_{n \ge 0} L(F)^n\right)  
   = \hat{\sigma}(L(F^*)).
   $$

Thus, the lemma holds.

**4. Theorem — Substitution Preserves Equivalence**
If $L(E_1) = L(E_2)$, then for all $R, S$:
$$
L\big(E_1[a \leftarrow R,\, b \leftarrow S]\big)  
= \hat{\sigma}(L(E_1))  
= \hat{\sigma}(L(E_2))  
= L\big(E_2[a \leftarrow R,\, b \leftarrow S]\big).
$$

Therefore, $E_1[a \leftarrow R,\, b \leftarrow S]$ and $E_2[a \leftarrow R,\, b \leftarrow S]$ are equivalent for all choices of $R, S$.

**Problem 4.20(ii)**

In particular, $$R=a+bb, S=ba^{*}$$

results in the complicated identity

$$((a+bb)^{*}(ba^{*}))^{*}(a+bb)^{*}=(a+bb)^{*}((ba^{*})(a+bb)^{*})^{*}$$

What identity would result from using $$R=(ba^{*})^{*}\quad S=(Λ+b)$$

**Answer 4.20(ii)**

$$(R^{*}S)^{*}R^{*}=R^{*}(SR^{*})^{*}$$
results in
$$((ba^{*})^{*}(Λ+b))^{*}(ba^{*})^{*}=(ba^{*})^{*}((Λ+b)(ba^{*})^{*})^{*}$$

---
**Chapter 5: Finite Automata**

---
**Problem 5.1:** Write out the transition tables for the FAs on pp. 56, 58 (both), 63, 64, and 69 that were defined by pictures.

**Answer 5.1:** *See below*

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_56'] = (\
  "$(a+b)^{*}b(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3]
  , [  '2',  1 ,  3]
  , [ '+3',  3 ,  3]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_56')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_1'] = (\
  "$(a+b)(a+b)^{*}$"
, SPAN('ab')
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  2 ,  2 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_1')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_2'] = (\
  "$(a+b)^{*}$"
, SPAN('ab') | ε()
, [ [  ' ', 'a', 'b']
  , ['-+1',  1 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_2')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_63'] = (\
  "$(a+b)^{*}(aa+bb)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + (σ('aa') | σ('bb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  4 ,  3 ]
  , [  '3',  2 ,  4 ]
  , [ '+4',  4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_63')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_64'] = (\
  "$(a+b)(a+b)b(a+b)*$"
, (σ('aaa') | σ('bbb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  5 ]
  , [  '4',  4 ,  4 ]
  , [ '+5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_64')

In [ ]:
# pg_69
na = nb = 0
def init(): na = nb = 0; return True
def acnt(): global na; na += 1; return True
def bcnt(): global nb; nb += 1; return True
FA['EVEN_EVEN'] = (\
  "$(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}$"
, POS(0) + Λ("init()")
+ ARBNO(
    σ('a') + λ("acnt()")
  | σ('b') + λ("bcnt()")
  )
+ RPOS(0) + λ("a % 2 == 0 and b % 2 == 0")
, [ [  ' ', 'a', 'b']
  , ['-+1',  3 ,  2 ]
  , [  '2',  4 ,  1 ]
  , [  '3',  1 ,  4 ]
  , [  '4',  2 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('EVEN_EVEN')

---
**Problem 5.2:** Build an FA that accepts only the language of all words with b as the second letter. Show both the picture and the transition table for this machine and find a regular expression for the language.

**Answer 5.2:**

RE = $(a + b)b(a+b)^{*}$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_2'] = (\
  "$(a+b)b(a+b)^{*}$"
, ANY("ab") + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  2 ]
  , [  '2', 3 ,  4 ]
  , [  '3', 3 ,  3 ]
  , [ '+4', 4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_2')

---
**Problem 5.3:** Build an FA that accepts only the words baa, ab, and abb and no other strings longer or shorter.

**Answer 5.3:**

RE = $(ab(Λ + b)+baa)$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_3'] = (\
  "$(ab(Λ + b)+baa)$"
, σ('baa') | σ('ab') | σ('abb')
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  5 ]
  , [  '2', 8 ,  3 ]
  , [ '+3', 8 ,  4 ]
  , [ '+4', 8 ,  8 ]
  , [  '5', 6 ,  8 ]
  , [  '6', 7 ,  8 ]
  , [ '+7', 8 ,  8 ]
  , [  '8', 8 ,  8 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_3')

---
**Problem 5.4:**
* (i) Build an FA with the three states that accepts all strings.
* (ii) Show that given any FA with three states and three *+*'s, it accepts all input strings.
* (iii) If an FA has three states and only one *+*, must it reject some inputs?

**Answer 5.4:**
- *See below*
- follows from the pigeon-hole principle
- No.

In [ ]:
FA['prob5_4'] = (\
  "$(a+b)(a+b)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b'))
, [ [ ' ' , 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  3 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_4')

---
**Problem 5.5:**
* (i) Build an FA that accepts only those words that have more that four letters.
* (ii) Build an FA that accepts only those words that have fewer than four letters.
* (iii) Build an FA that accepts only those words with exactly four letters.

**Answer 5.5:**
- RE: $\quad (a+b)^{5}(a+b)^{*}$
- RE: $\quad Λ+a+b+(Λ+a+b+(Λ+a+b)), \quad (Λ+a+b)^3$
- RE: $\quad (a+b)(a+b)(a+b)(a+b), \quad (a+b)^4$

In [ ]:
FA['prob5_5_1'] = (\
  "$(a+b)(a+b)(a+b)(a+b)(a+b)(a+b)^{*}$"
, SPAN("ab") @ "tx" + Λ("len(tx) > 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [  '5',  6 ,  6 ]
  , [ '+6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_1')

In [ ]:
FA['prob5_5_2'] = (\
  "$(Λ+a+b)(Λ+a+b)(Λ+a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) < 4")
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [ '+4',  5 ,  5 ]
  , [  '5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_2')

In [ ]:
FA['prob5_5_3'] = (\
  "$(a+b)(a+b)(a+b)(a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) == 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_3')

---
**Problem 5.6:** Build an FA that accepts only those words that do *not* end with *ba*.

**Answer 5.6:**
RE: $Λ+a+(a+b)^{*}(b+aa)$

In [ ]:
FA['prob5_6'] = (\
  "$Λ+a+(a+b)^{*}(b+aa)$"
, ARBNO(σ('a') + σ('b')) + σ('ba') + RPOS(0) + FAIL()
, [ [  ' ' , 'a', 'b']
  , ['-+1' ,  1 ,  2 ]
  , [ '+2' ,  3 ,  2 ]
  , [  '3' ,  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_6')

---
**Problem 5.7:** Build an FA that accepts only those words that begin or end with a double letter.

**Answer 5.7:** RE: $\quad (aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$

In [ ]:
FA['prob5_7'] = (\
  "$(aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$"
, ( POS(0) + (σ('aa') | σ('bb')) + ARB()
  | (σ('aa') | σ('bb')) + ARB() + RPOS(0)
  )
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  4 ]
  , [   '2',  3 ,  6 ]
  , [  '+3',  3 ,  3 ]
  , [   '4',  7 ,  5 ]
  , [  '+5',  5 ,  5 ]
  , [   '6',  7 ,  8 ]
  , [   '7',  9 ,  6 ]
  , [  '+8',  7 ,  8 ]
  , [  '+9',  9 ,  7 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_7')

---
**Problem 5.8:** Build an FA that accepts only those words that have an even number of substrings *ab*.

**Answer 5.8: ???**
- RE: $\quad (abab)^{*}$
- RE: $\quad (aa^{*}bb^{*}aa^{*}bb^{*})^{*}$

In [ ]:
FA['prob5_8_1'] = (\
  "$(abab)^{*}$"
, ARBNO(σ('abab'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  6 ]
  , [  '2',  6 ,  3 ]
  , [  '3',  4 ,  6 ]
  , [  '4',  6 ,  5 ]
  , [ '+5',  2 ,  6 ]
  , [  '6',  6 ,  6 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_8_1')

In [ ]:
FA['prob5_8_2'] = (\
  "$(aa^{*}bb^{*}aa^{*}bb^{*})^{*}$"
, ARBNO(
    σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  + σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  )
, [ [   ' ', 'a', 'b']
  , [ '-+1',  2 ,  1 ]
  , [   '2',  2 ,  3 ]
  , [   '3',  4 ,  3 ]
  , [   '4',  4 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_8_2')

---
**Problem 5.9:**
* (i) Recall from Chapter 4 the language of all words over the alphabet {*a b*} that have both the letter *a* and the letter *b* in them, but not necessarily in that order. Build an FA that accepts this language.
* (ii) Build an FA that accepts the language of all words with only *a*'s or only *b*'s in them. Give a regular expression for this language.

**Answer 5.9:**
- (i) RE: $(a+b)^{*}(ab+ba)(a+b)^{*}$ or $\quad (aa^{*}b+bb^{*}a)(a+b)^{*}$
- (ii) RE: $aa^{*}+bb^{*}$


In [ ]:
FA['prob5_9_1'] = (\
  "$(a+b)^{*}(ab+ba)(a+b)^{*}$"
, ARB() + σ('a') + ARB() + σ('b') + ARB()
| ARB() + σ('b') + ARB() + σ('a') + ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [ '+4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_1')

In [ ]:
FA['prob5_9_2'] = (\
  "$aa^{*}+bb^{*}$"
, SPAN("a") | SPAN("b")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [ '+2',  2 ,  4 ]
  , [ '+3',  4 ,  3 ]
  , [  '4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_2')

---
**Problem 5.10:** Consider all the possible FAs over the alphabet { a, b } that have exactly two states. An FA must have a designated start state, but there are four possible ways to place the *+*'s:

Type 1: ['-',  '']
Type 2: ['-+', '']
Type 3: ['-',  '+']
Type 4: ['-+', '+']

Each FA needs four edges (two from each state), each of which can lead to either of the states. There are $2^{4}=16$ ways to arrange the labeled edges for each of the four types of FAs. Therefore, in total there are 64 different FAs of two states. However, they do not represent 64 non-equivalent FAs because they are not all associated with different languages. All type 1 FAs do not accept any words at all, whereas all FAs of type 4 accepts all strings of *a*'s and *b*'s.

* (i) Draw the remaining FAs of type 2.
* (ii) Draw the remaining FAs of type 3.
* (iii) Recalculate the total number of two-state machines using the transition table definition.

**Book answer 5.10:**

* (i-ii) *See below*
* (iii) Transition tables for a two-state machine has four cells, two rows by two columns. Each may be filled with 1 or 2. So there are $2^4$ possibilities. There are still the same four ways to designate final states. Hence the total number of different transition tables is, surprise, 64, 2^4*4.

In [ ]:
import itertools
FA_template = (\
  ""
, ARB()
, [ [ ' ', 'a', 'b']
  , [ '1',  0 ,  0 ]
  , [ '2',  0 ,  0 ]])
#-------------------------------------------------------------------------------
for t in range(1,3):
    for i, combo in enumerate(itertools.product([1, 2], repeat=4)):
        name = f"prob_10_{i}"
        if t == 1:
            FA_template[2][1][0] = '-+1'
            FA_template[2][2][0] = '2'
        if t == 2:
            FA_template[2][1][0] = '-1'
            FA_template[2][2][0] = '+2'
        FA_template[2][1][1] = combo[0]
        FA_template[2][1][2] = combo[1]
        FA_template[2][2][1] = combo[2]
        FA_template[2][2][2] = combo[3]
        FA[name] = FA_template
        display_problem(name)

---
**Problem 5.11:** Show that there are exactly 5832 different finite automata with three states *x* *y*, *z* over the alphabet {*a b*}, where *x* is always the start state.

**Answer 5.11:** $1*2^3*3^6=5832$

---
**Problem 5.12:** Suppose a particular FA, called FIN, has the property that it had only one final state that was not the start state. During the night, vandals come and switch the *+* sign with the *-* sign and reverse the direction of all the edges.
* (i) Show that the picture that results might not actually be an FA at all by giving an example.
* (ii) Suppose, however, that in a particular case what resulted was, in fact, a perfectly good FA. Let us call it NIF. Give an example of one such machine.
* (iii) What is the relationship between the language accepted by FIN and the language accepted by NIF as described in part (ii)? Why?
* (iv) One of the vandals told me that if in FIN the plus state and the minus state were the same state, then the language accepted by the machine could contain only palindromic words. Defeat this vandal by example.

**Answer 5.12:**
* (i) Any FA with a state having more than one incoming of the same character is problematic
* (ii) Ensuring one of each in and out for each character of every state
* (iii) The language accepted by NIF is the reverse of every word in the language accepted by FIN. For every path from one state to another in FIN, the exact reverse path exists in NIF.

In [ ]:
FA['prob5_12_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  3 ]
  , [ '+3',  3 ,  3 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_1')

In [ ]:
FA['prob5_12_2'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [ '+3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_2')

In [ ]:
FA['prob5_12_3'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [  '3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_3')

---
**Problem 5.13:** We define a removable state as a state such that if we erase the state itself and the edges that come out of it, what results is a perfectly good-looking FA.
* (i) Give an example of an FA that contains a removable state.
* (ii) Show that if we erase a removable state the language defined by the reduced FA is exactly the same as the language defined by the old FA.

**Answer 5.13:**

* (i) *See below*
* (ii) An FA only retains its properties if the state being removed could not actually be reached, that is a state is removable only if there are no edges coming into it. Clearly if the stae is never used it has no bearing on the language that the FA accepts.

In [ ]:
FA['prob5_13_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  1 ,  1 ]
  , [ '+2',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_13_1')

---
**Problem 5.14:**
* (i) Build an FA that accepts the language of all strings of *a*'s and *b*'s such that the next to the last letter is an *a*.
* (ii) Build an FA that accepts the language of all strings of length 4 or more such that the next-to-last letter is equal to the second letter of the input string.

**Answer 5.14:** *See below*

In [ ]:
FA['prob5_14_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  1 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  3 ,  4 ]
  , [ '+4',  3 ,  1 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_14_1', verbose=True)

In [ ]:
FA['prob5_14_2'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  7 ]
  , [  '3',  4 ,  3 ]
  , [  '4',  5 ,  6 ]
  , [ '+5',  5 ,  6 ]
  , [ '+6',  5 ,  3 ]
  , [  '7',  7 ,  8 ]
  , [  '8', 10 ,  9 ]
  , [ '+9', 10 ,  9 ]
  , ['+10',  7 ,  9 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_14_2', verbose=True)

---
**Problem 5.15:** Build a machine that accepts all strings that have an even length that is not divisible by 6.

In [ ]:
FA['prob5_15'] = (\
  ""
, ARB
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  7 ,  7 ]
  , [  '7',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_15')

---
**Problem 5.16:** Build an FA such that when the labels *a* and *b* are swapped the new machine is different from the old one but equivalent (the language defined by these machines is the same).

**Answer 5.16:** *See below*

In [ ]:
FA['prob5_16'] = (\
  ""
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  3 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [  '4',  3 ,  3 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_16')

---
**Problem 5.17:**
* (i-iii) Describe in English the languages accepted by the following FAs.
* (iv) Write regular expressions for the languages accepted by these three machines.

**Answer 5.17:**
* (i) odd-length sequence ending in *a*
* (ii) non-null even-length sequence ending in *a*
* (iii) non-null even-length sequence of *xa*
* (iv) *See below*


In [ ]:
FA['prob5_17_1'] = (\
  "$(aa+ab+ba+bb)a$"
, ARB
, [ [  ' ', 'a', 'b']
  , [  '1',  2 ,  2 ]
  , [ '-2',  3 ,  1 ]
  , [ '+3',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_1')

In [ ]:
FA['prob5_17_2'] = (\
  "$(a+b)(aa+ab+ba+bb)a$"
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  2 ,  2 ]
  , [  '4',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_2')

In [ ]:
FA['prob5_17_3'] = (\
  "$(aa+ba)(aa+ba)^{*}$"
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  5 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  3 ,  5 ]
  , [  '5',  5 ,  5 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_3')

---
**Problem 5.18:** The following is an FA over the alphabet $\Sigma = \{ a\, b\, c \}$. Prove that it acepts all strings that have an odd number of ocurrences of the substring *abc*.

**Answer 5.18:** The machine will stay in states 1, 2, or 3 until *abc* is seen leading to state 4. The machine will stay in states 4, 5, or 6 until another *abc* is seen leading to state 1, and repeat. So no occurence of *abc* seen while in states 1-3, and one occurrence seen while in states 4-6. Due to repitition, this translates to even and odd. All states 4-6 are final states.

In [ ]:
FA['prob5_18'] = (\
  ""
, ARB
, [ [  ' ', 'a', 'b', 'c']
  , [ '-1',  2 ,  1 ,  1 ]
  , [  '2',  2 ,  3 ,  1 ]
  , [  '3',  2 ,  1 ,  4 ]
  , [ '+4',  5 ,  4 ,  4 ]
  , [ '+5',  5 ,  6 ,  4 ]
  , [ '+6',  5 ,  4 ,  1 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_18')

---
**Problem 5.19:** Consider the following FA:
* (i) Show that any input string with more than three letters is not accepted by this FA.
* (ii) Show that the only words accepted are *a*, *aab*, and *bab*.
* (iii) Show that by changing the location of *+* signs, we can make this FA accept the language { *bb* *aba* *bba* }.
* (iv) Show that any language in which the words have fewer than four letters can be accepted by a machine that looks like this one with the *+* signs in different places.
* (v) Prove that if L is a finite language, then there is some FA that accepts L extending the binary-tree part of this machine several more layers if necessary.

**Book answer 5.19:**
* (i) This FA starts out like a binary tree. For the first three letters of the input, each time we read a letter we reach some node on the next level. However reading any more letters traps us in the dead end state at the bottom. So no word of more than three letters can be accepted.
* (ii) There are exactly three final states in the machine. Concatenation of the labels on the transitions to these states shows that the machine accepts the words *a*, *aab*, and *bab*.
* (iii) Eliminate the three existing pluses. Place new pluses in each of the following states, the right-most node on level two, the third node from the left on level three, and the second node from the right also on level three.
* (iv) Each of the 15 possible strings with less than four letters is represented by one of the nodes (excluding the dead end state). Note the lexicographic order. Simply mark those nodes that represent the words in the language with plus signs.
* (v) Since L is finite, we can determine the longest word and then build a binary tree large enough to accommodate it. Next we add a dead end state with transitions from all the leaves and itself. For every word in L, we place a plus in the appropriate, unique state.

In [ ]:
FA['prob5_19'] = (\
  ""
, ARB
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  3 ]
  , [   '2',  4 ,  5 ]
  , [   '3',  6 ,  7 ]
  , [   '4',  8 ,  9 ]
  , [   '5', 10 , 11 ]
  , [   '6', 12 , 13 ]
  , [   '7', 14 , 15 ]
  , [   '8', 16 , 16 ]
  , [   '9', 16 , 16 ]
  , [  '10', 16 , 16 ]
  , [  '11', 16 , 16 ]
  , [  '12', 16 , 16 ]
  , [  '13', 16 , 16 ]
  , [  '14', 16 , 16 ]
  , [  '15', 16 , 16 ]
  , [  '16', 16 , 16 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_19')

---
**Problem 5.20:** Let us consider the possibility of an infinite automaton that starts with this infinite binary tree. Let L be any infinite language of strings of *a*'s and *b*s whatsoever. Show that by the judicious placement of *+*'s, we can turn the picture above into an infinite automaton to accept the language L. Show that for any given finite string, we can determine from this machine, in a finite time, whether it is a word in L. Discuss why this machine would not be a satisfactory language-definer for L.

**Book answer 5.20:** Even though the binary tree is infinite, the machine retains the uniqueness property (of the machines described in problem 19) that the path for every string ends in a different state. Surely just by placing the plus signs in the nodes that represent words in L would constitute an infinite automata. However, the machine obviously fails to define the language because no one can write down the infintely large picture.

---
**Chapter 6: Transition Graphs**

In [ ]:
import networkx as nx
from io import BytesIO
from PIL import Image
from IPython.display import display
from networkx.drawing.nx_agraph import to_agraph
TG = {}
def display_TG_problem(name):
    data = TG[name]
    starts, finals, states = [], set(), set()
    for code, trans in data:
        sid = code.lstrip('+-')
        states.add(sid)
        if code.startswith('-'): starts.append(sid)
        if code.startswith('+'): finals.add(sid)
        for tgt, _ in trans:
            states.add(str(tgt))
    G = nx.DiGraph()
    for sid in states:
        if sid in finals:
            G.add_node(sid, shape='doublecircle', color='white',
                penwidth='2', style='filled', fillcolor='white', fontcolor='black')
        else:
            G.add_node(sid, shape='circle', color='black',
                style='filled', fillcolor='white', fontcolor='black')
    edge_labels = {}
    for code, trans in data:
        src = code.lstrip('+-')
        for tgt, lbl in trans:
            key = (src, str(tgt))
            if key in edge_labels:
                edge_labels[key].append(lbl)
            else:
                edge_labels[key] = [lbl]
    for (src, tgt), labels in edge_labels.items():
        G.add_edge(src, tgt, label=", ".join(labels), color='white', fontcolor='white', fontsize='12')
    A = to_agraph(G)
    A.node_attr.update(style='filled', fillcolor='white', fontcolor='black')
    A.graph_attr.update(bgcolor='black', rankdir='LR')
    if starts:
        anchor = f'__start__{name}'
        A.add_node(anchor, label='', shape='none', width=0, height=0)
        for s in starts:
            A.add_edge(anchor, s, color='white')
    img = BytesIO()
    A.draw(img, format='png', prog='dot')
    img.seek(0)
    display(Image.open(img))

---
**Problem 6.1:** For each of the five FAs pictured in Problems 17, 19, and 20 in Chapter 5, build a transition graph that accepts the same language but has fewer states.

**Answer 6.1:**

---
**Problem 6.2:** For each of the next 10 words, decide which of the six machines on the next page accept the given word.
* (i) Λ
* (ii) *a*
* (iii) *b*
* (iv) *aa*
* (v) *ab*
* (vi) *aba*
* (vii) *abba*
* (viii) *bab*
* (ix) *baab*
* (x) *abbb*

**Answer 6.2:**

In [ ]:
TG['prob6_2_1'] = \
  [ [  '-1', [(2, "a"), (4, "ab"), (4, "b")]]
  , [   '2', [(2, "b"), (3, "Λ")]]
  , [   '3', [(3, "b"), (4, "a")]]
  , [  '+4', []]
  ]
#-------------------------------------------------------------------------------
TG['prob6_2_2'] = \
  [ [ '-+1', [(2, "a"), (3, "b")]]
  , [   '2', [(2, "a"), (3, "b")]]
  , [  '+3', []]
  ]
#-------------------------------------------------------------------------------
TG['prob6_2_3'] = \
  [ [  '-1', [(1, "ab"), (1, "ba"), (2, "aa"), (2, "bb")]]
  , [  '+2', [(2, "ab"), (2, "ba"), (2, "aa"), (2, "bb")]]
  ]
#-------------------------------------------------------------------------------
TG['prob6_2_4'] = \
  [ [  '-1', [(2, "a"), (3, "b")]]
  , [  '+2', [(1, "a"), (2, "b")]]
  , [  '+3', [(3, "a"), (1, "b")]]
  ]
#-------------------------------------------------------------------------------
TG['prob6_2_5'] = \
  [ [ '-+1', [(2, "Λ"), (5, "Λ")]]
  , [   '2', [(3, "a"), (2, "b")]]
  , [   '3', [(4, "a"), (3, "b")]]
  , [  '+4', [(4, "b")]]
  , [   '5', [(5, "a"), (6, "b")]]
  , [   '6', [(6, "a"), (7, "b")]]
  , [  '+7', [(7, "a")]]
  ]
#-------------------------------------------------------------------------------
TG['prob6_2_6'] = \
  [ [  '-1', [(2, "ab"), (4, "a")]]
  , [  '+2', [(3, "bb")]]
  , [  '+3', []]
  , [   '4', [(4, "b"), (3, "a")]]
  ]
#-------------------------------------------------------------------------------
display_TG_problem('prob6_2_1')
display_TG_problem('prob6_2_2')
display_TG_problem('prob6_2_3')
display_TG_problem('prob6_2_4')
display_TG_problem('prob6_2_5')
display_TG_problem('prob6_2_6')

---
**Problem 6.3:** Show that any language that can be accepted by a TG can be accepted by a TG with an even number of states.

**Answer 6.3:**

---
**Problem 6.4:** How many different TGs are there over the alphabet { *a*, *b* } That have two states?

**Answer 6.4:**

---
**Problem 6.5:** Prove that for every TG there is another TG that accepts the same language but has only one *+* state.

**Answer 6.5:**

---
**Problem 6.6:** Build a TG that accepts the language $L_1$ of all words that begin and end with the same double letter, either of the form *aa...aa* or *bb...bb*. Note: *aaa* and *bbb* are not words in the language.

**Answer 6.6:**

---
**Problem 6.7:** If OURSPONSER is a language that is accepted by a TG called Henry, prove that there is a TG that accepts the language of all strings of *a*'s and *b's that end in a word from OURSPONSER.

**Answer 6.7:**

---
**Problem 6.8:**
* (i) Suppose that $L$ is a finite language whose words are $w_1, w_2, w_3, ..., w_83.$ Prove that there is a TG that accepts exactly the language $L$.
* (ii) Of all TGs that accept exactly the language $L$, what is the one with the fewest number of states?

**Answer 6.8:**

---
**Problem 6.9:** Give a TG, called $TG_1$, that accepts the language $L_1$ and a TG, called $TG_2$ that accepts the language $L_2$, show how to build a new TG (called $TG_3$) that accepts exactly the language $L_1 + L_2$.

**Answer 6.9:**

---
**Problem 6.10:** Given $TG_1$ and $TG_2$ as described in Problem 6.9, show how to build $TG_4$ that accepts exactly the language $L_1L_2.$

**Answer 6.10:**

---
**Problem 6.11:**

**Answer 6.11:**

---
**Problem 6.12:**

**Answer 6.12:**

---
**Problem 6.13:**

**Answer 6.13:**

---
**Problem 6.14:**

**Answer 6.14:**

---
**Problem 6.15:**

**Answer 6.15:**

---
**Problem 6.16:**

**Answer 6.16:**

---
**Problem 6.17:**

**Answer 6.17:**

---
**Problem 6.18:**

**Answer 6.18:**

---
**Problem 6.19:**

**Answer 6.19:**

---
**Problem 6.20:**

**Answer 6.20:**